## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/rebeccaallison/Desktop/6_api/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Darnah,LY,32.7670,22.6367,62.56,75,92,8.86,overcast clouds,2022-04-30 19:28:51
1,1,Chifeng,CN,42.2683,118.9636,45.30,20,100,7.83,overcast clouds,2022-04-30 19:28:51
2,2,Okhotsk,RU,59.3833,143.3000,31.98,92,44,6.24,scattered clouds,2022-04-30 19:28:52
3,3,Vila Franca Do Campo,PT,37.7167,-25.4333,64.81,74,14,4.70,few clouds,2022-04-30 19:28:52
4,4,Guanica,PR,17.9716,-66.9080,81.61,93,100,9.69,overcast clouds,2022-04-30 19:28:53


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
4,4,Guanica,PR,17.9716,-66.9080,81.61,93,100,9.69,overcast clouds,2022-04-30 19:28:53
5,5,Rikitea,PF,-23.1203,-134.9692,75.87,75,53,12.26,light rain,2022-04-30 19:28:53
14,14,Vao,NC,-22.6667,167.4833,75.25,81,100,22.39,overcast clouds,2022-04-30 19:28:57
15,15,Omboue,GA,-1.5746,9.2618,79.56,90,100,6.91,overcast clouds,2022-04-30 19:28:58
24,24,Arraial Do Cabo,BR,-22.9661,-42.0278,76.86,78,75,5.75,broken clouds,2022-04-30 19:29:02
28,28,Butaritari,KI,3.0707,172.7902,80.53,83,100,13.18,overcast clouds,2022-04-30 19:29:05
30,30,Karratha,AU,-20.7377,116.8463,76.73,50,97,12.06,overcast clouds,2022-04-30 19:29:04
34,34,Vaini,TO,-21.2000,-175.2000,77.16,94,100,9.22,light intensity drizzle,2022-04-30 19:29:08
36,36,Georgetown,MY,5.4112,100.3354,82.35,89,20,3.44,few clouds,2022-04-30 19:29:09
38,38,Aransas Pass,US,27.9095,-97.1500,84.22,82,40,13.80,scattered clouds,2022-04-30 19:29:10


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
Date                   213
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Guanica,PR,81.61,overcast clouds,17.9716,-66.9080,
5,Rikitea,PF,75.87,light rain,-23.1203,-134.9692,
14,Vao,NC,75.25,overcast clouds,-22.6667,167.4833,
15,Omboue,GA,79.56,overcast clouds,-1.5746,9.2618,
24,Arraial Do Cabo,BR,76.86,broken clouds,-22.9661,-42.0278,
28,Butaritari,KI,80.53,overcast clouds,3.0707,172.7902,
30,Karratha,AU,76.73,overcast clouds,-20.7377,116.8463,
34,Vaini,TO,77.16,light intensity drizzle,-21.2000,-175.2000,
36,Georgetown,MY,82.35,few clouds,5.4112,100.3354,
38,Aransas Pass,US,84.22,scattered clouds,27.9095,-97.1500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Guanica,PR,81.61,overcast clouds,17.9716,-66.9080,Copamarina Beach Resort & Spa
5,Rikitea,PF,75.87,light rain,-23.1203,-134.9692,People ThankYou
14,Vao,NC,75.25,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
15,Omboue,GA,79.56,overcast clouds,-1.5746,9.2618,Hotel Olako
24,Arraial Do Cabo,BR,76.86,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
674,Necocli,CO,83.55,overcast clouds,8.4234,-76.7860,Hotel Boutique El Turista
677,Araguaina,BR,83.23,overcast clouds,-7.1911,-48.2072,Hotel São Vicente
678,Buraydah,SA,82.85,clear sky,26.3260,43.9750,Mövenpick Qassim
680,Morro Bay,US,85.24,clear sky,35.3658,-120.8499,Inn at Morro Bay


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/rebeccaallison/Desktop/6_api/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))